In [96]:
#Importing Pandas library for cleaning up raw data
import pandas as pd
import numpy as np

In [97]:
#File name of raw csv, which should be in the same folder/directory as the Preprocessing Notebook
raw_filename = 'clean_data.csv'

In [98]:
data = pd.read_csv(raw_filename, sep='\t')
data.head()

,Unnamed: 0,Country,I consider myself:,WiFi Router:Check all the internet connected devices you currently own:,Laptop computer:Check all the internet connected devices you currently own:,Smart phone:Check all the internet connected devices you currently own:,Smart TV:Check all the internet connected devices you currently own:,Activity Tracker (ex: Fitbit or Apple Watch):Check all the internet connected devices you currently own:,"Smarthome Hub (ex. Amazon Echo, Google Alexa):Check all the internet connected devices you currently own:",Car that connects to the internet:Check all the internet connected devices you currently own:,...,"Smart Appliance (ex. Coffeemaker, Refrigerator, Oven, Fridge):Check all the internet connected devices you currently own:",Smart Door Locks (ex. Door locks for your home you can open via bluetooth):Check all the internet connected devices you currently own:,"Smart Lighting (ex. Connected lighting switches, dimmers, or bulbs):Check all the internet connected devices you currently own:",Thinking about a future in which so much of your world is connected to the internet leaves you feeling:,What are you most excited about as we move toward a more digitally connected future?,What is your biggest fear as we move towards a more connected future?,Total Owned Devices,Country Average Devices,Country Average Tech Proficiency,Country Average Tech Sentiment
0,0,United States,3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,-2,None of the above,The loss of privacy,7.0,3.800052,2.605362,-0.071605
1,1,United States,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,-2,None of the above,We�۪ll lose touch with one another,6.0,3.800052,2.605362,-0.071605
2,2,United States,3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,How much fun it will be,I have no fears about a more connected future,4.0,3.800052,2.605362,-0.071605
3,3,United States,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1,How it will bring the world together,The loss of privacy,3.0,3.800052,2.605362,-0.071605
4,4,Canada,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,None of the above,The loss of privacy,3.0,3.513410,2.590333,-0.050987


In [121]:
#filter country by unique 
aggregate_data=data.drop_duplicates('Country')
aggregate_data.head()
aggregate_data=aggregate_data.drop('What is your biggest fear as we move towards a more connected future?',axis=1)

In [101]:
countryList=aggregate_data.Country.unique()
countryList

array(['United States', 'Canada', 'United Kingdom', 'France', 'India',
       'Mexico', 'Australia', 'Germany', 'Switzerland', 'Brazil',
       'Argentina', 'Italy', 'Belgium', 'Austria', 'Venezuela', 'Colombia',
       'Ecuador', 'Chile', 'Spain'], dtype=object)

In [159]:
aggregate_data['Count'] = np.nan

countryCounts=data['Country'].value_counts()
    
for country in countryList:
    aggregate_data.loc[aggregate_data.Country == country, ['Count']] = countryCounts[country]

aggregate_data


,Unnamed: 0,Country,I consider myself:,WiFi Router:Check all the internet connected devices you currently own:,Laptop computer:Check all the internet connected devices you currently own:,Smart phone:Check all the internet connected devices you currently own:,Smart TV:Check all the internet connected devices you currently own:,Activity Tracker (ex: Fitbit or Apple Watch):Check all the internet connected devices you currently own:,"Smarthome Hub (ex. Amazon Echo, Google Alexa):Check all the internet connected devices you currently own:",Car that connects to the internet:Check all the internet connected devices you currently own:,...,"Smart Appliance (ex. Coffeemaker, Refrigerator, Oven, Fridge):Check all the internet connected devices you currently own:",Smart Door Locks (ex. Door locks for your home you can open via bluetooth):Check all the internet connected devices you currently own:,"Smart Lighting (ex. Connected lighting switches, dimmers, or bulbs):Check all the internet connected devices you currently own:",Thinking about a future in which so much of your world is connected to the internet leaves you feeling:,What are you most excited about as we move toward a more digitally connected future?,Total Owned Devices,Country Average Devices,Country Average Tech Proficiency,Country Average Tech Sentiment,Count
0,0,United States,3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,-2,None of the above,7.0,3.800052,2.605362,-0.071605,15404.0
4,4,Canada,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,None of the above,3.0,3.513410,2.590333,-0.050987,3393.0
14,22,United Kingdom,4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,How much fun it will be,3.0,3.662828,2.641509,-0.094095,4081.0
16,24,France,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-2,None of the above,2.0,3.101538,2.591520,-0.251335,24907.0
21,35,India,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2,How it will make us all smarter and better edu...,3.0,3.471441,2.754957,0.580645,3379.0
25,44,Mexico,3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1,How easy it will make life,4.0,3.469156,2.549964,0.353896,5544.0
37,70,Australia,4,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1,How easy it will make life,5.0,3.458269,2.565060,-0.020218,1929.0
55,113,Germany,3,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1,How it will bring the world together,2.0,3.553127,2.607756,-0.105526,20630.0
88,192,Switzerland,4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1,How it will bring the world together,4.0,3.655559,2.636518,-0.081605,2941.0
93,204,Brazil,3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-1,How easy it will make life,4.0,3.543257,2.580833,0.520342,10299.0


In [102]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [184]:
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, TapTool, OpenURL
from bokeh.models import HoverTool
from bokeh.models import LinearInterpolator, CategoricalColorMapper
from bokeh.palettes import inferno
from bokeh.layouts import gridplot

size_mapper = LinearInterpolator(
    x = [data['Country'].value_counts().min(), data['Country'].value_counts().max()],
    y=[5,10]
)

color_mapper = CategoricalColorMapper(
    factors = countryList,
    palette = inferno(20)
)

source = ColumnDataSource(aggregate_data)

hover = HoverTool(tooltips = [
    ("records","@Count"),
    ("country" ,"@Country"),
], show_arrow=False)


plot1 = figure(
            title=str("Correlation Between Average Owned Devices and Sentiment of Future Technology for each Country with > 1000 survey responses"),
            x_axis_label='Average Sentiment of Future Techologies ((Negative) -2 to 2 (Positive))', y_axis_label='Average Number of Owned Devices',
            #x_range=(0,5),
            #y_range=(0,12)
            tools=[hover,"tap"]
)
plot1.circle(
        x='Country Average Tech Sentiment', y='Country Average Devices', 
        size= {'field': 'Count','transform':size_mapper},
        color= {'field': 'Country', 'transform':color_mapper}, 
        source=source
)


plot3 = figure(


)



url = "http://www.google.com/@Country/"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)


show(plot1)


In [179]:
plot2 = figure(
    title=str("Correlation Between Average Tech Knowledge and Sentiment of Future Technology for each Country with > 1000 survey responses"),
    x_axis_label='Average Sentiment of Future Techologies ((Negative) -2 to 2 (Positive))', y_axis_label='Average Number of Owned Devices',
    tools=[hover]
)

plot2.circle(
        x='Country Average Tech Sentiment', y='Country Average Tech Proficiency', 
        #size= {'field': 'I consider myself:','transform':size_mapper},
        color= {'field': 'Country', 'transform':color_mapper}, 
        source=source
)

plots = gridplot([plot1,plot2], ncols=2, plot_width=250, plot_height=250)
show(plots, notebook_handle=True)   

In [151]:
#Get column data source from data frame
source= ColumnDataSource(aggregate_data)
source.column_names

['Smart Door Locks (ex. Door locks for your home you can open via bluetooth):Check all the internet connected devices you currently own:',
 'Total Owned Devices',
 'Unnamed: 0',
 'Smart Thermostat (ex: Nest):Check all the internet connected devices you currently own:',
 'Country Average Devices',
 'Country',
 'Country Average Tech Proficiency',
 'Smart TV:Check all the internet connected devices you currently own:',
 'Smart Appliance (ex. Coffeemaker, Refrigerator, Oven, Fridge):Check all the internet connected devices you currently own:',
 'Smarthome Hub (ex. Amazon Echo, Google Alexa):Check all the internet connected devices you currently own:',
 'Country Average Tech Sentiment',
 'Smart Lighting (ex. Connected lighting switches, dimmers, or bulbs):Check all the internet connected devices you currently own:',
 'Laptop computer:Check all the internet connected devices you currently own:',
 'index',
 'WiFi Router:Check all the internet connected devices you currently own:',
 'Activity 

### Create variables for plotting

In [7]:
# x-axis values
x = ['2007','2008','2009','2010','2011','2012','2013','2014','2015']

#dictionary with the instituition column as the key and the values [0th index is Male Retention, 1th index is Female Retention 2nd index is Overall Retention]
plotDictionary = {}

def checkNaN(raw_val):
    if (pd.isnull(raw_val)):
        return float('nan')
    else:
        return float((1-raw_val)*100)

# create y values for (Male, Female, Overall Retention) for each school
for inst in institutions:
    #set 0 array for start    
    mRetention = 9*[0]
    fRetention = 9*[0]
    oRetention = 9*[0]
    
    #get rows of data for eachdd inst
    for year in x:
        #print("searching for year %s") %year
        for i,row in new_dataframe.loc[new_dataframe['Institution'].isin([inst]),:].iterrows():     
            if(int(year) == int(new_dataframe.loc[i]["School Year"][:4] )):
                #print(new_dataframe.loc[i,'Institution'])
                if(pd.isnull(new_dataframe.loc[i]["Male Retention"])):
                    mRetention[x.index(year)] = float('nan')
                    fRetention[x.index(year)] = float('nan')
                    oRetention[x.index(year)] = float('nan')
                    break
                else:
                    mRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Male Retention"])
                    fRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Female Retention"])
                    oRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Overall Retention"])
                    break
            #else:
                #print("cant find %s") %year
    #add retention % as value with institution as its key
    plotDictionary[inst] = mRetention, fRetention, oRetention
    

## Graphing Data

### Importing libraries

In [8]:
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
import itertools
from bokeh.models import ColumnDataSource
from bokeh.embed import file_html


output_notebook()

Loading BokehJS ...

In [9]:
# Method for cycling through color pallete
def color_gen():
    for c in itertools.cycle(Category10[10]):
        yield c
color = color_gen()

In [10]:
# #create a new plot
# plot1 = figure(
#     title="Retention Rates Among CS Students",
#     x_axis_label='Year', y_axis_label='Retention %',
#     y_range=(-5,5)
# )

# plot2 = figure(
#     title="Retention Rates Among CS Students",
#     x_axis_label='Year', y_axis_label='Retention %',
#     y_range=(-5,5)
# )

#draw lines
def drawLines(retentionFilter, inst):
    titleText=""
    if retentionFilter == 0: titleText='(Male)'    
    elif retentionFilter == 1: titleText='(Female)'
    elif retentionFilter == 2: titleText='(Male and Female)'
        
    ## Graph showing all retention rates for all filtered institutions    
    plot1 = figure(
        title="Retention Rates Among CS Students "+titleText,
        x_axis_label='Year', y_axis_label='Retention %',
        y_range=(-500,500)    
    )
    
    for institution in plotDictionary.keys():
        y = plotDictionary[institution][retentionFilter]   
        plot1.line(x, y, legend=str(institution), line_width=2, color=next(color))
    
    ## Graph showing all retention rates for selected institutions
    plot2 = figure(
    title="Retention Rates Among CS Students "+ titleText +" for Institution "+str(inst) ,
        x_axis_label='Year', y_axis_label='Retention %',
        y_range=(-500,500)
    )

    y2 = plotDictionary[inst][retentionFilter]   
    plot2.line(x, y2, legend=str(inst), line_width=2, color=next(color))

    plots = gridplot([[plot1,plot2], ncols=2, plot_width=250, plot_height=250])
    show(plots, notebook_handle=True)   

In [11]:
#callback method that hanldes graph updateswhen a change is made to the widget 
def update(FilterGender, ViewInstitution):
    if   FilterGender == "Male Retention": 
        drawLines(0, float(inst))
    elif FilterGender == "Female Retention": 
        drawLines(1,float(inst))
    elif FilterGender == "Overall Retention": 
        drawLines(2, float(inst))
    push_notebook()    

In [12]:
# Draw Graphs and control Widgets
interact(update, FilterGender=["Male Retention", "Female Retention", "Overall Retention"], ViewInstitution= map(str, institutions))
html = file_html(plots, CDN, "my plot")

A Jupyter Widget

NameError: name 'plots' is not defined

In [239]:
df = data.loc[data['Country']=='United States']
df

filterParam='Total Owned Devices'

cats = [-2,-1,0,-1,2]

# find the quartiles and IQR for each category
groups = df.groupby('Thinking about a future in which so much of your world is connected to the internet leaves you feeling:')
groups
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group[filterParam] > upper.loc[cat][filterParam]) | (group[filterParam] < lower.loc[cat][filterParam])][filterParam]
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for cat in cats:
        # only add outliers if they exist
        if not out.loc[cat].empty:
            for value in out[cat]:
                outx.append(cat)
                outy.append(value)

p = figure(tools="save", 
           #background_fill_color="#EFE8E2", 
           title="Distribution of "+ filterParam +" responses for each sentiment group",
           x_axis_label='Average Sentiment of Future Techologies ((Negative) -2 to 2 (Positive))', y_axis_label='Average Number of '+ filterParam,
)
#,x_range=['-2','-1','0','1','2']
# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper[filterParam] = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,filterParam]),upper[filterParam])]
lower[filterParam] = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,filterParam]),lower[filterParam])]

# stems
p.segment(cats, upper[filterParam], cats, q3[filterParam], line_color="black")
p.segment(cats, lower[filterParam], cats, q1[filterParam], line_color="black")

# boxes
p.vbar(cats, 0.7, q2[filterParam], q3[filterParam], fill_color="#3B8686", line_color="black")
p.vbar(cats, 0.7, q1[filterParam], q2[filterParam], fill_color="#3B8686", line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower[filterParam], 0.2, 0.01, line_color="black")
p.rect(cats, upper[filterParam], 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"

#output_file("boxplot.html", title="boxplot.py example")

show(p)

In [218]:
# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))


filterParam='score'

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat][filterParam]) | (group.score < lower.loc[cat][filterParam])][filterParam]
out = groups.apply(outliers).dropna()


group      
a      618    -3.460066
       782     2.625647
       1071    2.790948
b      24     -2.550304
       220     2.733948
       520     2.937566
       1168    3.167885
       1944   -2.757935
c      622     3.787555
       1431   -2.049960
       1906   -2.067253
d      224    -2.022511
       850     3.952952
       1033    3.600451
       1466   -1.558309
       1510   -1.579273
       1529    3.635922
       1936   -1.823841
e      1348   -0.821620
f      363    -0.812143
       1351   -0.754482
Name: score, dtype: float64

In [226]:
data['Thinking about a future in which so much of your world is connected to the internet leaves you feeling:'].unique()

array([-2,  1, -1,  0,  2], dtype=int64)